In [ ]:
import json
import pandas as pd
import numpy as np
import re

In [ ]:
file_dir = '/Users/henrybarze/Desktop/VDAB Projects/Modules/Module 8- ETL - Extract, Transform, Load/Module Files/Movies-ETL/Data'

In [ ]:
with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file: 
    wiki_movies_raw = json.load(file)

In [ ]:
len(wiki_movies_raw)

In [ ]:
# First 5 records
wiki_movies_raw[:5]

In [ ]:
# Last 5 records
wiki_movies_raw[-5:]

In [ ]:
# Some records in the middle
wiki_movies_raw[3600:3605]

In [ ]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)

In [ ]:
wiki_movies_df.head()

In [ ]:
sorted(wiki_movies_df.columns.tolist())

In [ ]:
wiki_movies = [movie for movie in wiki_movies_raw
              if ('Director' in movie or 'Directed by' in movie) 
                    and 'imdb_link' in movie
                    and 'No. of episodes' not in movie]
len(wiki_movies)


In [ ]:
def clean_movie(movie):
    movie_copy = dict(movie) #create a non-destructive copy
    return movie


In [ ]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']


In [ ]:
sorted(wiki_movies_df.columns.tolist())



In [ ]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] =movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
        
    # Merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie

In [ ]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

In [ ]:
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
print(len(wiki_movies_df))
wiki_movies_df.head()


In [ ]:
[[column,wiki_movies_df[column].isnull().sum()] for column in wiki_movies_df.columns]


In [ ]:
[column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]


In [ ]:
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]


In [ ]:
wiki_movies_df.dtypes

In [ ]:
box_office = wiki_movies_df['Box office'].dropna()

In [ ]:
def is_not_a_string(x):
    return type(x) != str

In [ ]:
box_office[box_office.map(lambda x: type(x) != str)]


In [ ]:
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)



In [ ]:
form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'
box_office.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()


In [ ]:
form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'
box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()

In [ ]:
matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE, na=False)

In [ ]:
box_office[~matches_form_one & ~matches_form_two]

In [ ]:
box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
